<a href="https://colab.research.google.com/github/trendinafrica/Comp_Neuro-ML_course/blob/main/Fixed_Points_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### Installing and importing relevant packages

In [2]:
# ! git clone https://github.com/mattgolub/fixed-point-finder.git
! git clone https://github.com/mattgolub/recurrent-whisperer.git

fatal: destination path 'recurrent-whisperer' already exists and is not an empty directory.


In [3]:
! git clone -b dependabot/pip/tensorflow-2.11.1 https://github.com/mattgolub/fixed-point-finder.git

fatal: destination path 'fixed-point-finder' already exists and is not an empty directory.


In [4]:
! pip install -r /content/fixed-point-finder/requirements-cpu.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining recurrent-whisperer from git+https://github.com/mattgolub/recurrent-whisperer.git@master#egg=recurrent-whisperer (from -r /content/fixed-point-finder/requirements-cpu.txt (line 7))
  Updating ./src/recurrent-whisperer clone (to revision master)
  Running command git fetch -q --tags
  Running command git reset --hard -q 4e0a0ab8e3655174132a96b910044bf867fb1a81
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.14.5.zip (4.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached scipy-1.1.0.tar.gz (15.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached scikit-learn-0.20.0.tar.gz (28.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached matplotlib-2.2.3.tar.gz (36.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached PyYAML-3.13.tar.gz (270 kB)
  Preparing metadata (setup.py) ... done
  Using cached tensorflow-2.11.1-cp310-cp310-manylinux

In [5]:
import sys
sys.path.insert(1, "/content/recurrent-whisperer")
sys.path.insert(1, "/content/fixed-point-finder")

In [6]:
import sys
print(sys.path)

['/content', '/content/fixed-point-finder', '/content/recurrent-whisperer', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.10/dist-packages/IPython/extensions', '/root/.ipython']


In [7]:
! pip install timer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() #added to enable v1 behavior LND
tf.enable_eager_execution()
# import tensorflow as tf

import numpy as np
import pdb
import sys
import argparse
from tensorflow.keras.optimizers.legacy import Adam

if os.environ.get('DISPLAY','') == '':
    # Ensures smooth running across environments, including servers without
    # graphical backends.
    print('No display found. Using non-interactive Agg backend.')
    import matplotlib
    matplotlib.use('Agg')

sys.path.insert(0, '/content/fixed-point-finder/')
sys.path.insert(0, "/content/recurrent-whisperer")

import matplotlib.pyplot as plt
# from FlipFlop import FlipFlop
from RecurrentWhisperer import RecurrentWhisperer
from FixedPointFinder import FixedPointFinder
from FixedPoints import FixedPoints
from plot_utils import plot_fps
import tf_utils

Instructions for updating:
non-resource variables are not supported in the long term


No display found. Using non-interactive Agg backend.
No display found. Using non-interactive Agg backend.
No display found. Using non-interactive Agg backend.


In [9]:
class FlipFlop(RecurrentWhisperer):
    ''' Class for training an RNN to implement an N-bit memory, a.k.a. "the
    flip-flop  task" as described in Sussillo & Barak, Neural Computation,
    2013.

    Task:
        Briefly, a set of inputs carry transient pulses (-1 or +1) to set the
        state of a set of binary outputs (also -1 or +1). Each input drives
        exactly one output. If the sign of an input pulse opposes the sign
        currently held at the corresponding output, the sign of the output
        flips. If an input pulse's sign matches that currently held at the
        corresponding output, the output does not change.

        This class generates synthetic data for the flip-flop task via
        generate_flipflop_trials(...).

    Usage:
        This class trains an RNN to generate the correct outputs given the
        inputs of the flip-flop task. All that is needed to get started is to
        construct a flipflop object and to call .train on that object:

        # dict of hyperparameter key/value pairs
        # (see 'Hyperparameters' section below)
        hps = {...}

        ff = FlipFlop(**hps)
        ff.train()

    Hyperparameters:
        rnn_type: string specifying the architecture of the RNN. Currently
        must be one of {'vanilla', 'gru', 'lstm'}. Default: 'vanilla'.

        n_hidden: int specifying the number of hidden units in the RNN.
        Default: 24.

        data_hps: dict containing hyperparameters for generating synthetic
        data. Contains the following keys:

            'n_batch': int specifying the number of synthetic trials to use
            per training batch (i.e., for one gradient step). Default: 128.

            'n_time': int specifying the duration of each synthetic trial
            (measured in timesteps). Default: 256.

            'n_bits': int specifying the number of input channels into the
            FlipFlop device (which will also be the number of output channels).
            Default: 3.

            'p_flip': float between 0.0 and 1.0 specifying the probability
            that a particular input channel at a particular timestep will
            contain a pulse (-1 or +1) on top of its steady-state value (0).
            Pulse signs are chosen by fair coin flips, and pulses are produced
            with the same statistics across all input channels and across all
            timesteps (i.e., there are no history effects, there are no
            interactions across input channels). Default: 0.2.

        log_dir: string specifying the top-level directory for saving various
        training runs (where each training run is specified by a different set
        of hyperparameter settings). When tuning hyperparameters, log_dir is
        meant to be constant across models. Default: '/tmp/flipflop_logs/'.

        n_trials_plot: int specifying the number of synthetic trials to plot
        per visualization update. Default: 4.
    '''

    @staticmethod
    def _default_hash_hyperparameters():
        '''Defines default hyperparameters, specific to FlipFlop, for the set
        of hyperparameters that are hashed to define a directory structure for
        easily managing multiple runs of the RNN training (i.e., using
        different hyperparameter settings). Additional default hyperparameters
        are defined in RecurrentWhisperer (from which FlipFlop inherits).

        Args:
            None.

        Returns:
            dict of hyperparameters.
        '''
        return {
            'rnn_type': 'vanilla',
            'n_hidden': 24,
            'data_hps': {
                'n_batch': 128,
                'n_time': 256,
                'n_bits': 3,
                'p_flip': 0.2}
            }

    @staticmethod
    def _default_non_hash_hyperparameters():
        '''Defines default hyperparameters, specific to FlipFlop, for the set
        of hyperparameters that are NOT hashed. Additional default
        hyperparameters are defined in RecurrentWhisperer (from which FlipFlop
        inherits).

        Args:
            None.

        Returns:
            dict of hyperparameters.
        '''
        return {
            'log_dir': '/tmp/flipflop_logs/',
            'n_trials_plot': 1,
            }

    def _setup_model(self):
        '''Defines an RNN in Tensorflow.

        See docstring in RecurrentWhisperer.

        '''

        hps = self.hps
        n_hidden = hps.n_hidden

        data_hps = hps.data_hps
        n_batch = data_hps['n_batch']
        n_time = data_hps['n_time']
        n_inputs = data_hps['n_bits']
        n_output = n_inputs

        # Data handling
        self.inputs_bxtxd = tf.placeholder(tf.float32,
            [n_batch, n_time, n_inputs])
        self.output_bxtxd = tf.placeholder(tf.float32,
            [n_batch, n_time, n_output])

        # RNN
        if hps.rnn_type == 'vanilla':
            self.rnn_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        elif hps.rnn_type == 'gru':
            self.rnn_cell = tf.nn.rnn_cell.GRUCell(n_hidden)
        elif hps.rnn_type == 'lstm':
            self.rnn_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
        else:
            raise ValueError('Hyperparameter rnn_type must be one of '
                '[vanilla, gru, lstm] but was %s' % hps.rnn_type)

        initial_state = self.rnn_cell.zero_state(n_batch, dtype=tf.float32)

        if hps.rnn_type == 'lstm':
            self.state_bxtxd = tf_utils.unroll_LSTM(
                self.rnn_cell,
                inputs=self.inputs_bxtxd,
                initial_state=initial_state)

            self.hidden_bxtxd = self.state_bxtxd.h

        else:
            self.state_bxtxd, _ = tf.nn.dynamic_rnn(
                self.rnn_cell,
                inputs=self.inputs_bxtxd,
                initial_state=initial_state)

            self.hidden_bxtxd = self.state_bxtxd

        # Readout from RNN
        np_W_out, np_b_out = self._np_init_weight_matrix(n_hidden, n_output)
        self.W_out = tf.Variable(np_W_out, dtype=tf.float32, name='W_out')
        self.b_out = tf.Variable(np_b_out, dtype=tf.float32, name='b_out')
        self.pred_output_bxtxd = tf.tensordot(self.hidden_bxtxd,
            self.W_out, axes=1) + self.b_out

        # Loss
        self.loss = tf.reduce_mean(
            tf.squared_difference(self.output_bxtxd, self.pred_output_bxtxd))

    def _np_init_weight_matrix(self, input_size, output_size):
        '''Randomly initializes a weight matrix W and bias vector b.

        For use with input data matrix X [n x input_size] and output data
        matrix Y [n x output_size], such that Y = X*W + b (with broadcast
        addition). This is the typical required usage for TF dynamic_rnn.

        Weights drawn from a standard normal distribution and are then
        rescaled to preserve input-output variance.

        Args:
            input_size: non-negative int specifying the number of input
            dimensions of the linear mapping.

            output_size: non-negative int specifying the number of output
            dimensions of the linear mapping.

        Returns:
            W: numpy array of shape [input_size x output_size] containing
            randomly initialized weights.

            b: numpy array of shape [output_size,] containing all zeros.
        '''
        if input_size == 0:
            scale = 1.0 # This avoids divide by zero error
        else:
            scale = 1.0 / np.sqrt(input_size)
        W = np.multiply(scale,self.rng.randn(input_size, output_size))
        b = np.zeros(output_size)
        return W, b

    def _build_data_feed_dict(self, data, **kwargs):
        '''Performs a training step over a single batch of data.

        Args:
            data: dict containing one epoch of data. Contains the
            following key/value pairs:

                'inputs': [n_batch x n_time x n_bits] numpy array specifying
                the inputs to the RNN.

                'outputs': [n_batch x n_time x n_bits] numpy array specifying
                the correct output responses to the 'inputs.'

        Returns:
            dict with (TF placeholder, feed value) as (key, value) pairs.
        '''
        feed_dict = dict()
        feed_dict[self.inputs_bxtxd] = data['inputs']
        feed_dict[self.output_bxtxd] = data['output']
        return feed_dict

    def _get_pred_ops(self):
        ''' See docstring in RecurrentWhisperer._get_pred_ops()
        '''

        return {
            'state': self.state_bxtxd,
            'output': self.pred_output_bxtxd
            }

    def _get_batch_size(self, batch_data):
        '''See docstring in RecurrentWhisperer.'''
        return batch_data['inputs'].shape[0]

    def generate_data(self, train_or_valid_str=None):
        '''Generates synthetic data (i.e., ground truth trials) for the
        FlipFlop task. See comments following FlipFlop class definition for a
        description of the input-output relationship in the task.

        Args:
            None (RecurrentWhisperer option train_or_valid_str is ignored).

        Returns:
            dict containing 'inputs' and 'outputs'.

                'inputs': [n_batch x n_time x n_bits] numpy array containing
                input pulses.

                'outputs': [n_batch x n_time x n_bits] numpy array specifying
                the correct behavior of the FlipFlop memory device.
        '''

        data_hps = self.hps.data_hps
        n_batch = data_hps['n_batch']
        n_time = data_hps['n_time']
        n_bits = data_hps['n_bits']
        p_flip = data_hps['p_flip']

        # Randomly generate unsigned input pulses
        unsigned_inputs = self.rng.binomial(
            1, p_flip, [n_batch, n_time, n_bits])

        # Ensure every trial is initialized with a pulse at time 0
        unsigned_inputs[:, 0, :] = 1

        # Generate random signs {-1, +1}
        random_signs = 2*self.rng.binomial(
            1, 0.5, [n_batch, n_time, n_bits]) - 1

        # Apply random signs to input pulses
        inputs = np.multiply(unsigned_inputs, random_signs)

        # Allocate output
        output = np.zeros([n_batch, n_time, n_bits])

        # Update inputs (zero-out random start holds) & compute output
        for trial_idx in range(n_batch):
            for bit_idx in range(n_bits):
                input_ = np.squeeze(inputs[trial_idx, :, bit_idx])
                t_flip = np.where(input_ != 0)
                for flip_idx in range(np.size(t_flip)):
                    # Get the time of the next flip
                    t_flip_i = t_flip[0][flip_idx]

                    '''Set the output to the sign of the flip for the
                    remainder of the trial. Future flips will overwrite future
                    output'''
                    output[trial_idx, t_flip_i:, bit_idx] = \
                        inputs[trial_idx, t_flip_i, bit_idx]

        return {'inputs': inputs, 'output': output}

    def _split_data_into_batches(self, data):
        '''See docstring in RecurrentWhisperer.'''

        # Just use a single batch in this simple example.
        return [data], None

    def _combine_prediction_batches(self, pred_list, summary_list, idx_list):
        '''See docstring in RecurrentWhisperer.'''

        # Just use a single batch in this simple example.

        assert (len(pred_list)==1),\
            ('FlipFlop only supports single batches, but found %d batches.'
             % len(pred_list))

        assert (len(summary_list)==1),\
            ('FlipFlop only supports single batches, but found %d batches.'
             % len(summary_list))

        return pred_list[0], summary_list[0]

    def _update_visualizations(self, data, pred,
        train_or_valid_str=None,
        version=None):
        '''See docstring in RecurrentWhisperer.'''

        self.plot_trials(data, pred)
        self.refresh_figs()

    def plot_trials(self, data, pred, start_time=0, stop_time=None):
        '''Plots example trials, complete with input pulses, correct outputs,
        and RNN-predicted outputs.

        Args:
            data: dict as returned by generate_flipflop_trials.

            start_time (optional): int specifying the first timestep to plot.
            Default: 0.

            stop_time (optional): int specifying the last timestep to plot.
            Default: n_time.

        Returns:
            None.
        '''

        FIG_WIDTH = 6 # inches
        FIG_HEIGHT = 3 # inches

        fig = self._get_fig('example_trials',
            width=FIG_WIDTH,
            height=FIG_HEIGHT)

        hps = self.hps
        n_batch = self.hps.data_hps['n_batch']
        n_time = self.hps.data_hps['n_time']
        n_plot = np.min([hps.n_trials_plot, n_batch])

        inputs = data['inputs']
        output = data['output']
        pred_output = pred['output']

        if stop_time is None:
            stop_time = n_time

        time_idx = range(start_time, stop_time)

        for trial_idx in range(n_plot):
            ax = plt.subplot(n_plot, 1, trial_idx+1)
            if n_plot == 1:
                plt.title('Example trial', fontweight='bold')
            else:
                plt.title('Example trial %d' % (trial_idx + 1),
                          fontweight='bold')

            self._plot_single_trial(
                inputs[trial_idx, time_idx, :],
                output[trial_idx, time_idx, :],
                pred_output[trial_idx, time_idx, :])

            # Only plot x-axis ticks and labels on the bottom subplot
            if trial_idx < (n_plot-1):
                plt.xticks([])
            else:
                plt.xlabel('Timestep', fontweight='bold')

    @staticmethod
    def _plot_single_trial(input_txd, output_txd, pred_output_txd):

        VERTICAL_SPACING = 2.5
        [n_time, n_bits] = input_txd.shape
        tt = range(n_time)

        y_ticks = [VERTICAL_SPACING*bit_idx for bit_idx in range(n_bits)]
        y_tick_labels = \
            ['Bit %d' % (n_bits-bit_idx) for bit_idx in range(n_bits)]

        plt.yticks(y_ticks, y_tick_labels, fontweight='bold')
        for bit_idx in range(n_bits):

            vertical_offset = VERTICAL_SPACING*bit_idx

            # Input pulses
            plt.fill_between(
                tt,
                vertical_offset + input_txd[:, bit_idx],
                vertical_offset,
                step='mid',
                color='gray')

            # Correct outputs
            plt.step(
                tt,
                vertical_offset + output_txd[:, bit_idx],
                where='mid',
                linewidth=2,
                color='cyan')

            # RNN outputs
            plt.step(
                tt,
                vertical_offset + pred_output_txd[:, bit_idx],
                where='mid',
                color='purple',
                linewidth=1.5,
                linestyle='--')

        plt.xlim(-1, n_time)

In [10]:
def train_FlipFlop(train_mode):
    ''' Train an RNN to solve the N-bit memory task.

        Args:
            train_mode: 1, 2, or 3.

                1.  Generate on-the-fly training data (new data for each
                    gradient step)
                2.  Provide a single, fixed set of training data.
                3.  Provide, single, fixed set of training data (as in 2) and
                    a single, fixed set of validation data.

                (see docstring to RecurrentWhisperer.train() for more detail)

        Returns:
            model: FlipFlop object.

                The trained RNN model.

            valid_predictions: dict.

                The model's predictions on a set of held-out validation trials.
    '''

    assert train_mode in [1, 2, 3], \
        ('train_mode must be 1, 2, or 3, but was %s' % str(train_mode))

    # Hyperparameters for FlipFlop
    # See FlipFlop.py for detailed descriptions.
    hps = {
            'rnn_type': 'lstm',
            'n_hidden': 16,
            'min_loss': 1e-4,
            'log_dir': './logs/',
            'do_generate_pretraining_visualizations': True,

            'data_hps': {
                'n_batch': 512,
                'n_time': 64,
                'n_bits': 3,
                'p_flip': 0.5
                },

            # Hyperparameters for AdaptiveLearningRate
            'alr_hps': {
                'initial_rate': 1.0,
                'min_rate': 1e-5
                }
            }

    model = FlipFlop(**hps)
    train_data = model.generate_data()
    valid_data = model.generate_data()

    if train_mode == 1:
        model.train()
    elif train_mode == 2:
        # This runs much faster at the expense of overfitting potential
        model.train(train_data)
    elif train_mode == 3:
        # This requires some changes to hps to fully leverage validation)
        model.train(train_data, valid_data)

    # Get example state trajectories from the network
    # Visualize inputs, outputs, and RNN predictions from example trials
    valid_predictions, valid_summary = model.predict(valid_data)
    model.plot_trials(valid_data, valid_predictions)

    return model, valid_predictions

def find_fixed_points(model, valid_predictions):
    ''' Find, analyze, and visualize the fixed points of the trained RNN.

    Args:
        model: FlipFlop object.

            Trained RNN model, as returned by train_FlipFlop().

        valid_predictions: dict.

            Model predictions on validation trials, as returned by
            train_FlipFlop().

    Returns:
        None.
    '''

    '''Initial states are sampled from states observed during realistic
    behavior of the network. Because a well-trained network transitions
    instantaneously from one stable state to another, observed networks states
    spend little if any time near the unstable fixed points. In order to
    identify ALL fixed points, noise must be added to the initial states
    before handing them to the fixed point finder. In this example, the noise
    needed is rather large, which can lead to identifying fixed points well
    outside of the domain of states observed in realistic behavior of the
    network--such fixed points can be safely ignored when interpreting the
    dynamical landscape (but can throw visualizations).'''

    NOISE_SCALE = 0.5 # Standard deviation of noise added to initial states
    N_INITS = 1024 # The number of initial states to provide

    n_bits = model.hps.data_hps['n_bits']
    is_lstm = model.hps.rnn_type == 'lstm'

    '''Fixed point finder hyperparameters. See FixedPointFinder.py for detailed
    descriptions of available hyperparameters.'''
    fpf_hps = {}

    # Setup the fixed point finder
    fpf = FixedPointFinder(model.rnn_cell, model.session, **fpf_hps)

    # Study the system in the absence of input pulses (e.g., all inputs are 0)
    inputs = np.zeros([1,n_bits])

    '''Draw random, noise corrupted samples of those state trajectories
    to use as initial states for the fixed point optimizations.'''
    initial_states = fpf.sample_states(valid_predictions['state'],
        n_inits=N_INITS,
        noise_scale=NOISE_SCALE)

    # Run the fixed point finder
    unique_fps, all_fps = fpf.find_fixed_points(initial_states, inputs)

    # Visualize identified fixed points with overlaid RNN state trajectories
    # All visualized in the 3D PCA space fit the the example RNN states.
    plot_fps(unique_fps, valid_predictions['state'],
        plot_batch_idx=range(30),
        plot_start_time=10)

    print('Entering debug mode to allow interaction with objects and figures.')
    print('You should see a figure with:')
    print('\tMany blue lines approximately outlining a cube')
    print('\tStable fixed points (black dots) at corners of the cube')
    print('\tUnstable fixed points (red lines or crosses) '
        'on edges, surfaces and center of the cube')
    print('Enter q to quit.\n')
    pdb.set_trace()

def main():

    parser = argparse.ArgumentParser(
        description='FixedPointFinder: Flip Flop example')
    parser.add_argument('--train_mode', default=1, type=int)
    args = vars(parser.parse_args())
    train_mode = args['train_mode']

    # Step 1: Train an RNN to solve the N-bit memory task
    model, valid_predictions = train_FlipFlop(train_mode)

    # STEP 2: Find, analyze, and visualize the fixed points of the trained RNN
    find_fixed_points(model, valid_predictions)

# if __name__ == '__main__':
#     main()


In [11]:
  # parser = argparse.ArgumentParser(
  #     description='FixedPointFinder: Flip Flop example')
  # parser.add_argument('--train_mode', default=1, type=int)
  # args = vars(parser.parse_args())
  # train_mode = args['train_mode']

  # Step 1: Train an RNN to solve the N-bit memory task
  model, valid_predictions = train_FlipFlop(2)

  # STEP 2: Find, analyze, and visualize the fixed points of the trained RNN
  find_fixed_points(model, valid_predictions)

<ipython-input-9-89a6ee66214d>:136: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  self.rnn_cell = tf.nn.rnn_cell.LSTMCell(n_hidden)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



Run directory found: ./logs/06862fd7da.
No checkpoints found.
Attempting to build TF model on gpu:0

Placing CPU-only ops on cpu:0



AttributeError: ignored